# PlantSpy 3- Modele Combine final
## A- Espece plantes
## B- DetectionMaladie (toutes plante sauf Tomato, Apple, Corn, Grapes, Potato)
## C- definir le type de maladie/Healthy (seulement pour Tomato, Apple, Corn, Grape, Potato) 

## 0- Importer les librairies et connection au Drive

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import os
import pathlib
from PIL import Image 
from sklearn.model_selection import train_test_split
%matplotlib inline

from matplotlib import cm

from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import Sequential
from tensorflow.keras import callbacks

from tensorflow.keras.applications.vgg16 import VGG16


from sklearn import metrics # Pour évaluer les modèles

import itertools # Pour créer des iterateurs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1- Data Processing

### Dictionnaires Type de maladie par especes

In [ ]:
# Grape
dict_maladie_Grape1 = { 0: 'Black_rot' ,
                 1: 'Esca_(Black_Measles)' ,
                 2 : 'Leaf_blight_(Isariopsis_Leaf_Spot)',
                 3: 'healthy'}

list_maladie_Grape1 =[ 'Black_rot' ,'Black_Measles' , 'Leaf_blight','healthy' ]

dict_maladie_Grape2 = { 0: 'Black_rot' ,
                 1: 'Esca_(Black_Measles)' ,
                 2 : 'Leaf_blight_(Isariopsis_Leaf_Spot)'}

list_maladie_Grape2 =[ 'Black_rot' ,'Black_Measles' , 'Leaf_blight' ]

#POTATO

dict_maladie_Potato1 = { 0: 'Early_blight' ,
                 1: 'Late_blight' ,
                 2: 'healthy' }

list_maladie_Potato1 =['Early_blight', 'Late_blight', 'healthy']

dict_maladie_Potato2 = { 0: 'Early_blight' ,
                 1: 'Late_blight' }

list_maladie_Potato2 =['Early_blight', 'Late_blight']

#TOMATO

dict_maladie_Tomato1 = { 0: 'Bacterial_spot' ,
                 1: 'Early_blight' ,
                 2 : 'Late_blight' ,
                 3 : 'Leaf_Mold',
                 4: 'Septoria_leaf_spot' ,
                 5: 'Spider_mites Two-spotted_spider_mite' ,
                 6: 'Target_Spot' ,
                 7 : 'Tomato_Yellow_Leaf_Curl_Virus' ,
                 8 : 'Tomato_mosaic_virus',
                 9 : 'healthy'}

list_maladie_Tomato1 = [ 'Bacterial_spot' ,'Early_blight', 'Late_blight' ,'Leaf_Mold','Septoria_leaf_spot' , 'Spider_mites Two-spotted_spider_mite' ,
                'Target_Spot' , 'Tomato_Yellow_Leaf_Curl_Virus' , 'Tomato_mosaic_virus', 'healthy'  ]

dict_maladie_Tomato2 = { 0: 'Bacterial_spot' ,
                 1: 'Early_blight' ,
                 2 : 'Late_blight' ,
                 3 : 'Leaf_Mold',
                 4: 'Septoria_leaf_spot' ,
                 5: 'Spider_mites Two-spotted_spider_mite' ,
                 6: 'Target_Spot' ,
                 7 : 'Tomato_Yellow_Leaf_Curl_Virus' ,
                 8 : 'Tomato_mosaic_virus',
                 9 : 'healthy'}

list_maladie_Tomato2 = [ 'Bacterial_spot' ,'Early_blight', 'Late_blight' ,'Leaf_Mold','Septoria_leaf_spot' , 'Spider_mites Two-spotted_spider_mite' ,
                'Target_Spot' , 'Tomato_Yellow_Leaf_Curl_Virus' , 'Tomato_mosaic_virus', 'healthy'  ]


# APPLE

dict_maladie_Apple1 = { 0: 'Apple_scab' ,
                 1: 'Black_rot' ,
                 2 : 'Cedar_apple_rust',
                3: 'healthy'}

list_maladie_Apple1 =[ 'Apple_scab' ,'Black_rot' ,'Cedar_apple_rust','healthy' ]

dict_maladie_Apple2 = { 0: 'Apple_scab' ,
                 1: 'Black_rot' ,
                 2 : 'Cedar_apple_rust'}

list_maladie_Apple2 =[ 'Apple_scab' ,'Black_rot' ,'Cedar_apple_rust' ]


# CORN

dict_maladie_Corn1 = { 2: 'Northern_Leaf_Blight' ,
                 1: 'Common_rust_' ,
                 0 : 'Cercospora_leaf_spot Gray_leaf_spot',
                 3: 'healthy'}

list_maladie_Corn1 =[ 'Cercospora_leaf_spot Gray_leaf_spot', 'Common_rust_' , 'Northern_Leaf_Blight','healthy' ]

dict_maladie_Corn2 = { 2: 'Northern_Leaf_Blight' ,
                 1: 'Common_rust_' ,
                 0 : 'Cercospora_leaf_spot Gray_leaf_spot'}

list_maladie_Corn2 =[ 'Cercospora_leaf_spot Gray_leaf_spot', 'Common_rust_' , 'Northern_Leaf_Blight']

### Recuperation des donnees




#### Valid Dataset

In [ ]:
folder_path = "/content/drive/MyDrive/NewPlantDiseasesDataset/valid/"

X_valid0=[]
y_valid_categorie=[]
y_valid_typeMaladie=[]
y_valid_healthy=[]

width = 100
height = 100 # resize to 100x100 to solve the RAM issue in Kaggle notebooks

for path, dirs, files in os.walk(folder_path):
    for repertoire in dirs:
      chemin=folder_path+repertoire
      for images in os.listdir(chemin):
        img=cv2.imread(chemin+'/'+images,cv2.IMREAD_COLOR )
        img_resized = cv2.resize(img, (width,height))
        X_valid0.append(np.array(img_resized))
        healthy=("healthy" in repertoire)*1 
        y_valid_categorie.append(repertoire.split('___')[0])   # On recupere la catégorie 
        y_valid_typeMaladie.append(repertoire.split('___')[1])   # On recupere le type de maladie seulement 
        y_valid_healthy.append(healthy)  # Detection maladie ou healthy





#### Data processing

In [ ]:
X_valid0= np.array(X_valid0)
X_valid0 = X_valid0.astype('float32')
X_valid = X_valid0 / 255
print(X_valid.shape)

X_valid= np.array(X_valid0)/255
X_valid = X_valid.astype('float32')

y_valid_categorie0 = np.array(y_valid_categorie)
y_valid_typeMaladie0 = np.array(y_valid_typeMaladie)
y_valid_healthy0 = np.array(y_valid_healthy)

(17572, 100, 100, 3)


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoderCat =  LabelEncoder()
y_valid_categorie = encoderCat.fit_transform(y_valid_categorie0)



# A - Classification : Especes Plantes

#### load modele classification par categorie de Plantes

In [ ]:
model_categorie = tf.keras.models.load_model('/content/drive/MyDrive/model_LeNet1_Categorie_AllData_Softmax')


In [ ]:
predict_categorie = model_categorie.predict(X_valid)


In [ ]:
predict_categorie_class = predict_categorie.argmax(axis = 1)

predict_categorie_label = encoderCat.inverse_transform(predict_categorie_class)
y_valid_categorie_label = encoderCat.inverse_transform(y_valid_categorie)

print(metrics.classification_report(y_valid_categorie_label, predict_categorie_label))

                         precision    recall  f1-score   support

                  Apple       0.94      0.96      0.95      1943
              Blueberry       0.96      0.96      0.96       454
Cherry_(including_sour)       0.98      0.95      0.96       877
           Corn_(maize)       0.99      0.98      0.98      1829
                  Grape       1.00      0.98      0.99      1805
                 Orange       0.99      0.96      0.98       503
                  Peach       0.98      0.89      0.93       891
           Pepper,_bell       0.97      0.86      0.92       975
                 Potato       0.89      0.99      0.93      1426
              Raspberry       0.98      0.96      0.97       445
                Soybean       0.94      0.94      0.94       505
                 Squash       0.96      0.99      0.98       434
             Strawberry       0.97      0.98      0.98       900
                 Tomato       0.96      0.98      0.97      4585

               accuracy

In [ ]:
df_predict_categorie_label = pd.DataFrame(predict_categorie_label, columns=["predicted_categorie"])
df_predict_categorie_label.value_counts()

predicted_categorie    
Tomato                     4670
Apple                      1973
Corn_(maize)               1795
Grape                      1775
Potato                     1583
Strawberry                  914
Pepper,_bell                865
Cherry_(including_sour)     850
Peach                       809
Soybean                     510
Orange                      489
Blueberry                   454
Squash                      449
Raspberry                   436
dtype: int64

In [ ]:
actual_categorie=pd.DataFrame(y_valid_categorie_label, columns=["categorie_reelle"])
df_finalA=pd.concat([ actual_categorie,df_predict_categorie_label],axis=1)
pd.crosstab(df_finalA['categorie_reelle'],df_finalA['predicted_categorie'])

predicted_categorie,Apple,Blueberry,Cherry_(including_sour),Corn_(maize),Grape,Orange,Peach,"Pepper,_bell",Potato,Raspberry,Soybean,Squash,Strawberry,Tomato
categorie_reelle,,,,,,,,,,,,,,
Apple,1858,8,7,1,0,0,5,3,20,0,2,2,3,34
Blueberry,8,434,0,0,0,0,1,1,1,0,0,0,3,6
Cherry_(including_sour),7,1,833,0,0,0,0,6,13,0,8,3,1,5
Corn_(maize),13,0,0,1784,2,1,0,0,8,0,1,8,4,8
Grape,1,0,0,0,1768,0,0,1,6,4,8,1,8,8
Orange,2,0,5,0,0,485,0,0,2,0,0,0,0,9
Peach,46,1,1,0,1,0,789,5,5,0,0,1,0,42
"Pepper,_bell",11,4,3,0,0,0,4,842,37,2,10,0,3,59
Potato,7,2,0,0,0,0,0,0,1406,0,0,0,1,10


# B- Detection Malade ou Saine

### 2 - Recuperation du modele healthy/UnHealthy

In [ ]:
model_healthy = tf.keras.models.load_model("/content/drive/MyDrive/model_isHealthy_18K")

In [ ]:
# Nouvelles predictions du modele
pred_healthy=model_healthy.predict(X_valid)


In [ ]:
pred_healthy_class= pd.DataFrame(pred_healthy.round().astype(int), columns=['Is_healthy_pred'])
actual_healthy =  (pd.DataFrame((y_valid_healthy0), columns=['Is_healthy_reelle']))

In [ ]:
actual_healthy.value_counts()

Is_healthy_reelle
0                    12000
1                     5572
dtype: int64

In [ ]:
pred_healthy_class.value_counts()

Is_healthy_pred
0                  10644
1                   6928
dtype: int64

In [ ]:
print(metrics.classification_report(actual_healthy, pred_healthy_class))

              precision    recall  f1-score   support

           0       1.00      0.88      0.94     12000
           1       0.80      0.99      0.88      5572

    accuracy                           0.92     17572
   macro avg       0.90      0.94      0.91     17572
weighted avg       0.93      0.92      0.92     17572



In [ ]:
df_finalB=pd.concat([actual_healthy, pred_healthy_class],axis=1)
pd.crosstab(df_finalB['Is_healthy_reelle'],df_finalB['Is_healthy_pred'])

Is_healthy_pred,0,1
Is_healthy_reelle,,
0,10591,1409
1,53,5519


In [ ]:
df_final=pd.concat([df_finalA,df_finalB], axis=1)

df_final.head()

,categorie_reelle,predicted_categorie,Is_healthy_reelle,Is_healthy_pred
0,Apple,Apple,0,0
1,Apple,Apple,0,0
2,Apple,Apple,0,0
3,Apple,Squash,0,0
4,Apple,Apple,0,0


In [ ]:
pd.crosstab(df_final['Is_healthy_reelle'],df_final['Is_healthy_pred'])

Is_healthy_pred,0,1
Is_healthy_reelle,,
0,10591,1409
1,53,5519


In [ ]:
df_final['Maladie_reelle']=pd.DataFrame(y_valid_typeMaladie)
df_final['pred_maladie']="healthy"

df_final.loc[df_final.Is_healthy_pred==0,'pred_maladie']='malade'



In [ ]:
pd.crosstab(df_final['pred_maladie'],df_final['Maladie_reelle'])

Maladie_reelle,Apple_scab,Bacterial_spot,Black_rot,Cedar_apple_rust,Cercospora_leaf_spot Gray_leaf_spot,Common_rust_,Early_blight,Esca_(Black_Measles),Haunglongbing_(Citrus_greening),Late_blight,...,Leaf_blight_(Isariopsis_Leaf_Spot),Leaf_scorch,Northern_Leaf_Blight,Powdery_mildew,Septoria_leaf_spot,Spider_mites Two-spotted_spider_mite,Target_Spot,Tomato_Yellow_Leaf_Curl_Virus,Tomato_mosaic_virus,healthy
pred_maladie,,,,,,,,,,,,,,,,,,,,,
healthy,84,83,69,43,2,3,26,3,45,57,...,24,2,2,137,33,251,273,64,157,5519
malade,420,1279,900,397,408,474,939,477,458,891,...,406,442,475,718,403,184,184,426,291,53


In [ ]:
df_final.head()

,categorie_reelle,predicted_categorie,Is_healthy_reelle,Is_healthy_pred,Maladie_reelle,pred_maladie
0,Apple,Apple,0,0,Apple_scab,malade
1,Apple,Apple,0,0,Apple_scab,malade
2,Apple,Apple,0,0,Apple_scab,malade
3,Apple,Squash,0,0,Apple_scab,malade
4,Apple,Apple,0,0,Apple_scab,malade


# C - Classification Type de Maladie (Apple, Corn, Grape, Potato et Tomato)

## C.1 - Predict type maladie de "Potato"

In [ ]:
model_maladiePotato = tf.keras.models.load_model('/content/drive/MyDrive/model_Potato1_avecHealthy')


In [ ]:
# liste des maladies "Potato"
y_valid_typeMaladie_Potato = y_valid_typeMaladie0[y_valid_categorie0=="Potato"]
print(pd.DataFrame(y_valid_typeMaladie_Potato).value_counts())

Early_blight    485
Late_blight     485
healthy         456
dtype: int64


In [ ]:
X_img_predicted_Potato  = X_valid[(df_final['predicted_categorie'] == "Potato")]

X_img_predicted_Potato.shape

(1583, 100, 100, 3)

In [ ]:
predict_typeMaladiePotato = model_maladiePotato.predict(X_img_predicted_Potato)


In [ ]:
predict_typeMaladiePotato_class = predict_typeMaladiePotato.argmax(axis = 1)

In [ ]:
predict_typeMaladiePotato_label = (pd.DataFrame(predict_typeMaladiePotato_class, columns=['maladie_pred'])).replace({"maladie_pred": dict_maladie_Potato1})


In [ ]:
pd.DataFrame(predict_typeMaladiePotato_label).value_counts()


maladie_pred
healthy         536
Early_blight    535
Late_blight     512
dtype: int64

In [ ]:
predict_typeMaladiePotato_label.head()

,maladie_pred
0,Late_blight
1,healthy
2,Late_blight
3,Late_blight
4,Late_blight


In [ ]:
df_final_Potato=df_final[(df_final['predicted_categorie'] == "Potato") ].copy()
df_final_Potato=df_final_Potato.reset_index()
df_final_Potato['pred_maladie']=predict_typeMaladiePotato_label['maladie_pred']
df_final_Potato = df_final_Potato.rename(index = lambda x: "Potato_"+ str(x))
df_final_Potato =df_final_Potato .drop(['index'], axis=1)

In [ ]:
df_final_Potato.head()

,categorie_reelle,predicted_categorie,Is_healthy_reelle,Is_healthy_pred,Maladie_reelle,pred_maladie
Potato_0,Apple,Potato,0,0,Apple_scab,Late_blight
Potato_1,Apple,Potato,0,0,Apple_scab,healthy
Potato_2,Apple,Potato,0,0,Apple_scab,Late_blight
Potato_3,Apple,Potato,0,0,Apple_scab,Late_blight
Potato_4,Apple,Potato,0,0,Apple_scab,Late_blight


In [ ]:
pd.crosstab(df_final_Potato['pred_maladie'],df_final_Potato['Maladie_reelle'])

Maladie_reelle,Apple_scab,Bacterial_spot,Black_rot,Cedar_apple_rust,Cercospora_leaf_spot Gray_leaf_spot,Early_blight,Haunglongbing_(Citrus_greening),Late_blight,Leaf_blight_(Isariopsis_Leaf_Spot),Leaf_scorch,Northern_Leaf_Blight,Powdery_mildew,Septoria_leaf_spot,Target_Spot,healthy
pred_maladie,,,,,,,,,,,,,,,
Early_blight,3,3,1,0,4,483,0,31,0,2,3,0,4,0,1
Late_blight,10,8,1,0,0,7,2,473,1,0,1,4,0,2,3
healthy,1,6,0,1,0,1,0,3,0,1,0,4,1,2,516


## C.2 Predict type maladie de "Tomato"

In [ ]:
model_maladieTomato = tf.keras.models.load_model('/content/drive/MyDrive/model_Tomate_avecHealthy_softmax')


In [ ]:
# liste des maladies "Tomato"
y_valid_typeMaladie_Tomato = y_valid_typeMaladie0[y_valid_categorie0=="Tomato"]
print(pd.DataFrame(y_valid_typeMaladie_Tomato).value_counts())

Tomato_Yellow_Leaf_Curl_Virus           490
healthy                                 481
Early_blight                            480
Leaf_Mold                               470
Late_blight                             463
Target_Spot                             457
Tomato_mosaic_virus                     448
Septoria_leaf_spot                      436
Spider_mites Two-spotted_spider_mite    435
Bacterial_spot                          425
dtype: int64


In [ ]:
X_img_predicted_Tomato  = X_valid[(df_final['predicted_categorie'] == "Tomato")]

In [ ]:
predict_typeMaladieTomato = model_maladieTomato.predict(X_img_predicted_Tomato)


In [ ]:
predict_typeMaladieTomato_class = predict_typeMaladieTomato.argmax(axis = 1)

In [ ]:
predict_typeMaladieTomato_label = (pd.DataFrame(predict_typeMaladieTomato_class, columns=['maladie_pred'])).replace({"maladie_pred": dict_maladie_Tomato1})


In [ ]:
pd.DataFrame(predict_typeMaladieTomato_label).value_counts()

maladie_pred                        
Early_blight                            658
healthy                                 624
Tomato_mosaic_virus                     484
Tomato_Yellow_Leaf_Curl_Virus           454
Target_Spot                             438
Spider_mites Two-spotted_spider_mite    432
Leaf_Mold                               410
Bacterial_spot                          409
Late_blight                             404
Septoria_leaf_spot                      357
dtype: int64

In [ ]:
df_final_Tomato=df_final[(df_final['predicted_categorie'] == "Tomato")].copy()
df_final_Tomato=df_final_Tomato.reset_index()
df_final_Tomato['pred_maladie']=predict_typeMaladieTomato_label['maladie_pred']
df_final_Tomato= df_final_Tomato.rename(index = lambda x: "Tomato_"+ str(x))
df_final_Tomato =df_final_Tomato.drop(['index'], axis=1)

In [ ]:
df_final_Tomato.head()

,categorie_reelle,predicted_categorie,Is_healthy_reelle,Is_healthy_pred,Maladie_reelle,pred_maladie
Tomato_0,Apple,Tomato,0,0,Apple_scab,Late_blight
Tomato_1,Apple,Tomato,0,0,Apple_scab,Late_blight
Tomato_2,Apple,Tomato,0,0,Apple_scab,Late_blight
Tomato_3,Apple,Tomato,0,0,Apple_scab,Leaf_Mold
Tomato_4,Apple,Tomato,0,0,Apple_scab,Early_blight


In [ ]:
pd.crosstab(df_final_Tomato['pred_maladie'],df_final_Tomato['Maladie_reelle'])

Maladie_reelle,Apple_scab,Bacterial_spot,Black_rot,Cedar_apple_rust,Cercospora_leaf_spot Gray_leaf_spot,Common_rust_,Early_blight,Haunglongbing_(Citrus_greening),Late_blight,Leaf_Mold,Leaf_blight_(Isariopsis_Leaf_Spot),Leaf_scorch,Northern_Leaf_Blight,Powdery_mildew,Septoria_leaf_spot,Spider_mites Two-spotted_spider_mite,Target_Spot,Tomato_Yellow_Leaf_Curl_Virus,Tomato_mosaic_virus,healthy
pred_maladie,,,,,,,,,,,,,,,,,,,,
Bacterial_spot,0,378,0,7,2,0,0,3,1,0,0,1,0,1,3,0,1,11,0,1
Early_blight,1,60,2,1,0,0,438,0,49,38,1,3,1,0,37,5,0,8,0,14
Late_blight,3,8,0,1,1,1,3,0,358,4,1,2,2,1,12,0,1,4,0,2
Leaf_Mold,1,3,0,0,0,0,0,0,2,399,0,0,0,0,5,0,0,0,0,0
Septoria_leaf_spot,1,19,2,1,0,0,3,4,0,8,0,0,0,0,317,0,0,0,0,2
Spider_mites Two-spotted_spider_mite,0,15,0,0,0,0,2,0,2,6,0,0,0,1,5,357,28,14,1,1
Target_Spot,0,7,0,0,0,0,5,0,0,0,0,0,0,2,11,26,364,3,7,13
Tomato_Yellow_Leaf_Curl_Virus,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,450,0,0
Tomato_mosaic_virus,0,1,0,1,0,0,1,0,0,8,0,0,0,0,18,8,5,0,436,6


## C.3 Predict type maladie de "Apple"

In [ ]:
model_maladieApple = tf.keras.models.load_model('/content/drive/MyDrive/model_Apple_Healthy')


In [ ]:
# liste des maladies "Apple"
y_valid_typeMaladie_Apple = y_valid_typeMaladie0[y_valid_categorie0=="Apple"]
print(pd.DataFrame(y_valid_typeMaladie_Apple).value_counts())

Apple_scab          504
healthy             502
Black_rot           497
Cedar_apple_rust    440
dtype: int64


In [ ]:
X_img_predicted_Apple  = X_valid[(df_final['predicted_categorie'] == "Apple")]

X_img_predicted_Apple.shape

(1973, 100, 100, 3)

In [ ]:
predict_typeMaladieApple = model_maladieApple.predict(X_img_predicted_Apple)


In [ ]:
predict_typeMaladieApple_class = predict_typeMaladieApple.argmax(axis = 1)

In [ ]:
predict_typeMaladieApple_label = (pd.DataFrame(predict_typeMaladieApple_class, columns=['maladie_pred'])).replace({"maladie_pred": dict_maladie_Apple1})


In [ ]:
pd.DataFrame(predict_typeMaladieApple_label).value_counts()


maladie_pred    
Black_rot           602
healthy             493
Apple_scab          464
Cedar_apple_rust    414
dtype: int64

In [ ]:
predict_typeMaladieApple_label.head()

,maladie_pred
0,Apple_scab
1,Apple_scab
2,Apple_scab
3,Apple_scab
4,healthy


In [ ]:
df_final_Apple=df_final[(df_final['predicted_categorie'] == "Apple")].copy()
df_final_Apple=df_final_Apple.reset_index()
df_final_Apple['pred_maladie']=predict_typeMaladieApple_label['maladie_pred']
df_final_Apple= df_final_Apple.rename(index = lambda x: "Apple_"+ str(x))
df_final_Apple =df_final_Apple.drop(['index'], axis=1)

In [ ]:
pd.crosstab(df_final_Apple['pred_maladie'],df_final_Apple['Maladie_reelle'])

Maladie_reelle,Apple_scab,Bacterial_spot,Black_rot,Cedar_apple_rust,Cercospora_leaf_spot Gray_leaf_spot,Early_blight,Esca_(Black_Measles),Haunglongbing_(Citrus_greening),Late_blight,Leaf_Mold,Northern_Leaf_Blight,Powdery_mildew,Septoria_leaf_spot,Target_Spot,healthy
pred_maladie,,,,,,,,,,,,,,,
Apple_scab,410,3,27,1,2,0,0,0,0,1,1,1,0,1,17
Black_rot,45,21,464,7,2,2,1,0,0,0,1,1,2,1,55
Cedar_apple_rust,5,2,0,399,4,0,0,0,0,0,1,1,0,0,2
healthy,18,4,0,4,2,9,0,2,2,1,0,1,0,0,450


##C.4 Predict type maladie de "Corn

In [ ]:
model_maladieCorn = tf.keras.models.load_model('/content/drive/MyDrive/model_Corn_Healthy')


In [ ]:
# liste des maladies "Corn"
y_valid_typeMaladie_Corn = y_valid_typeMaladie0[y_valid_categorie0=="Corn_(maize)"]
print(pd.DataFrame(y_valid_typeMaladie_Corn).value_counts())

Common_rust_                           477
Northern_Leaf_Blight                   477
healthy                                465
Cercospora_leaf_spot Gray_leaf_spot    410
dtype: int64


In [ ]:
X_img_predicted_Corn  = X_valid[(df_final['predicted_categorie'] == "Corn_(maize)")]

X_img_predicted_Corn.shape

(1795, 100, 100, 3)

In [ ]:
predict_typeMaladieCorn = model_maladieCorn.predict(X_img_predicted_Corn)


In [ ]:
predict_typeMaladieCorn_class = predict_typeMaladieCorn.argmax(axis = 1)

In [ ]:
predict_typeMaladieCorn_label = (pd.DataFrame(predict_typeMaladieCorn_class, columns=['maladie_pred'])).replace({"maladie_pred": dict_maladie_Corn1})


In [ ]:
pd.DataFrame(predict_typeMaladieCorn_label).value_counts()


maladie_pred                       
Cercospora_leaf_spot Gray_leaf_spot    629
healthy                                438
Northern_Leaf_Blight                   431
Common_rust_                           297
dtype: int64

In [ ]:
predict_typeMaladieCorn_label.head()

,maladie_pred
0,Cercospora_leaf_spot Gray_leaf_spot
1,Northern_Leaf_Blight
2,Cercospora_leaf_spot Gray_leaf_spot
3,Cercospora_leaf_spot Gray_leaf_spot
4,Cercospora_leaf_spot Gray_leaf_spot


In [ ]:
df_final_Corn=df_final[(df_final['predicted_categorie'] == "Corn_(maize)")].copy()
df_final_Corn=df_final_Corn.reset_index()
df_final_Corn['pred_maladie']=predict_typeMaladieCorn_label['maladie_pred']
df_final_Corn= df_final_Corn.rename(index = lambda x: "Corn_"+ str(x))
df_final_Corn =df_final_Corn.drop(['index'], axis=1)

In [ ]:
pd.crosstab(df_final_Corn['pred_maladie'],df_final_Corn['Maladie_reelle'])

Maladie_reelle,Bacterial_spot,Cedar_apple_rust,Cercospora_leaf_spot Gray_leaf_spot,Common_rust_,Late_blight,Leaf_scorch,Northern_Leaf_Blight,Powdery_mildew,healthy
pred_maladie,,,,,,,,,
Cercospora_leaf_spot Gray_leaf_spot,1,1,366,185,5,1,55,1,14
Common_rust_,0,0,0,291,1,0,1,0,4
Northern_Leaf_Blight,0,0,20,0,1,0,408,0,2
healthy,0,0,0,0,0,0,1,0,437


## C.5 Predict type maladie de "Grape

In [ ]:
model_maladieGrape = tf.keras.models.load_model('/content/drive/MyDrive/model_Grape1_Healthy')


In [ ]:
# liste des maladies "Grape"
y_valid_typeMaladie_Grape = y_valid_typeMaladie0[y_valid_categorie0=="Grape"]
print(pd.DataFrame(y_valid_typeMaladie_Grape).value_counts())

Esca_(Black_Measles)                  480
Black_rot                             472
Leaf_blight_(Isariopsis_Leaf_Spot)    430
healthy                               423
dtype: int64


In [ ]:
X_img_predicted_Grape  = X_valid[(df_final['predicted_categorie'] == "Grape")]

X_img_predicted_Grape.shape

(1775, 100, 100, 3)

In [ ]:
predict_typeMaladieGrape = model_maladieGrape.predict(X_img_predicted_Grape)


In [ ]:
predict_typeMaladieGrape_class = predict_typeMaladieGrape.argmax(axis = 1)

In [ ]:
predict_typeMaladieGrape_label = (pd.DataFrame(predict_typeMaladieGrape_class, columns=['maladie_pred'])).replace({"maladie_pred": dict_maladie_Grape1})


In [ ]:
pd.DataFrame(predict_typeMaladieGrape_label).value_counts()


maladie_pred                      
Esca_(Black_Measles)                  500
Black_rot                             442
Leaf_blight_(Isariopsis_Leaf_Spot)    433
healthy                               400
dtype: int64

In [ ]:
predict_typeMaladieGrape_label.head()

,maladie_pred
0,Esca_(Black_Measles)
1,Black_rot
2,Black_rot
3,Black_rot
4,Black_rot


In [ ]:
df_final_Grape=df_final[(df_final['predicted_categorie'] == "Grape")].copy()
df_final_Grape=df_final_Grape.reset_index()
df_final_Grape['pred_maladie']=predict_typeMaladieGrape_label['maladie_pred']
df_final_Grape= df_final_Grape.rename(index = lambda x: "Grape_"+ str(x))
df_final_Grape =df_final_Grape.drop(['index'], axis=1)

In [ ]:
pd.crosstab(df_final_Grape['pred_maladie'],df_final_Grape['Maladie_reelle'])

Maladie_reelle,Bacterial_spot,Black_rot,Cercospora_leaf_spot Gray_leaf_spot,Early_blight,Esca_(Black_Measles),Leaf_blight_(Isariopsis_Leaf_Spot),Septoria_leaf_spot,healthy
pred_maladie,,,,,,,,
Black_rot,1,437,1,0,3,0,0,0
Esca_(Black_Measles),1,22,1,1,475,0,0,0
Leaf_blight_(Isariopsis_Leaf_Spot),1,7,0,0,1,423,1,0
healthy,0,1,0,0,0,1,0,398


# 3- Regroupement de toutes les predictions

In [ ]:
df_finalC=pd.concat([df_final_Potato,df_final_Tomato,df_final_Apple,df_final_Grape, df_final_Corn], axis=0)
list_C=["Tomato","Potato","Grape", "Corn_(maize)", "Apple"]


In [ ]:
df_finalD=df_final[(~df_final['predicted_categorie'].isin(list_C))].copy() # toutes les autres categorie de plantes

df_finalD.loc[df_finalD.Is_healthy_reelle==0,'Maladie_reelle']='malade' # pour les autres categorie on ne predit que la presence de maladie


In [ ]:
df_actual_pred=pd.concat([df_finalC,df_finalD], axis=0)
df_actual_pred[df_actual_pred['categorie_reelle']=='Squash'].head()

,categorie_reelle,predicted_categorie,Is_healthy_reelle,Is_healthy_pred,Maladie_reelle,pred_maladie
Potato_1525,Squash,Potato,0,0,Powdery_mildew,Late_blight
Tomato_187,Squash,Tomato,0,0,Powdery_mildew,Bacterial_spot
Corn_1785,Squash,Corn_(maize),0,0,Powdery_mildew,Cercospora_leaf_spot Gray_leaf_spot
11653,Squash,Squash,0,0,malade,malade
11654,Squash,Squash,0,0,malade,malade


In [ ]:
print(metrics.classification_report(df_actual_pred['categorie_reelle'],df_actual_pred['predicted_categorie']))

                         precision    recall  f1-score   support

                  Apple       0.94      0.96      0.95      1943
              Blueberry       0.96      0.96      0.96       454
Cherry_(including_sour)       0.98      0.95      0.96       877
           Corn_(maize)       0.99      0.98      0.98      1829
                  Grape       1.00      0.98      0.99      1805
                 Orange       0.99      0.96      0.98       503
                  Peach       0.98      0.89      0.93       891
           Pepper,_bell       0.97      0.86      0.92       975
                 Potato       0.89      0.99      0.93      1426
              Raspberry       0.98      0.96      0.97       445
                Soybean       0.94      0.94      0.94       505
                 Squash       0.96      0.99      0.98       434
             Strawberry       0.97      0.98      0.98       900
                 Tomato       0.96      0.98      0.97      4585

               accuracy

In [ ]:
print(metrics.classification_report(df_actual_pred['Is_healthy_reelle'],df_actual_pred['Is_healthy_pred']))

              precision    recall  f1-score   support

           0       1.00      0.88      0.94     12000
           1       0.80      0.99      0.88      5572

    accuracy                           0.92     17572
   macro avg       0.90      0.94      0.91     17572
weighted avg       0.93      0.92      0.92     17572



In [ ]:
print(metrics.classification_report(df_actual_pred['Maladie_reelle'],df_actual_pred['pred_maladie']))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                      precision    recall  f1-score   support

                          Apple_scab       0.88      0.82      0.85       499
                      Bacterial_spot       0.92      0.69      0.79       548
                           Black_rot       0.86      0.93      0.90       964
                    Cedar_apple_rust       0.96      0.93      0.95       429
 Cercospora_leaf_spot Gray_leaf_spot       0.58      0.90      0.71       406
                        Common_rust_       0.98      0.61      0.75       477
                        Early_blight       0.77      0.96      0.86       956
                Esca_(Black_Measles)       0.95      0.99      0.97       480
     Haunglongbing_(Citrus_greening)       0.00      0.00      0.00        13
                         Late_blight       0.91      0.88      0.90       939
                           Leaf_Mold       0.97      0.85      0.91       470
  Leaf_blight_(Isariopsis_Leaf_Spot)       0.98      0.99      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df_actual_pred['erreur_pred_ModeleA']=1-(df_actual_pred['categorie_reelle']==df_actual_pred['predicted_categorie'])*1
df_actual_pred['erreur_pred_ModeleB']=1-(df_actual_pred['Is_healthy_reelle']==df_actual_pred['Is_healthy_pred'])*1
df_actual_pred['erreur_pred_ModeleC']=1-(df_actual_pred['Maladie_reelle']==df_actual_pred['pred_maladie'])*1




In [ ]:
#erreurs Total
df_actual_pred['erreur_pred_tot']=1-((1-df_actual_pred['erreur_pred_ModeleA'])*(1-df_actual_pred['erreur_pred_ModeleC']))


In [ ]:
# acuuracy Modele Classification Especes
print(" # bonnes predictions:", df_actual_pred['erreur_pred_tot'].count()-df_actual_pred['erreur_pred_ModeleA'].sum())
print(" % bonnes predictions:", (df_actual_pred['erreur_pred_tot'].count()-df_actual_pred['erreur_pred_ModeleA'].sum())/df_actual_pred['erreur_pred_tot'].count())

 # bonnes predictions: 16896
 % bonnes predictions: 0.961529706351013


In [ ]:
# accuracy Groupe 1
df_groupe_1=df_actual_pred[~df_actual_pred['categorie_reelle'].isin(list_C)]

print(" # bonnes predictions:", df_groupe_1['erreur_pred_tot'].count()-df_groupe_1['erreur_pred_ModeleB'].sum())
print(" % bonnes predictions:", (df_groupe_1['erreur_pred_tot'].count()-df_groupe_1['erreur_pred_ModeleB'].sum())/df_groupe_1['erreur_pred_tot'].count())

 # bonnes predictions: 5745
 % bonnes predictions: 0.9600601604278075


In [ ]:
df_groupe_1['categorie_reelle'].value_counts()

Pepper,_bell               975
Strawberry                 900
Peach                      891
Cherry_(including_sour)    877
Soybean                    505
Orange                     503
Blueberry                  454
Raspberry                  445
Squash                     434
Name: categorie_reelle, dtype: int64

In [ ]:
df_groupe_1.groupby('categorie_reelle')['erreur_pred_ModeleB'].sum()

categorie_reelle
Blueberry                   3
Cherry_(including_sour)    98
Orange                     45
Peach                      20
Pepper,_bell               32
Raspberry                   0
Soybean                     0
Squash                     39
Strawberry                  2
Name: erreur_pred_ModeleB, dtype: int64

In [ ]:
# accuracy Groupe 2
df_groupe_2=df_actual_pred[df_actual_pred['categorie_reelle'].isin(list_C)]

print(" # bonnes predictions:", df_groupe_2['erreur_pred_tot'].count()-df_groupe_2['erreur_pred_ModeleC'].sum())
print(" % bonnes predictions:", (df_groupe_2['erreur_pred_tot'].count()-df_groupe_2['erreur_pred_ModeleC'].sum())/df_groupe_2['erreur_pred_tot'].count())

 # bonnes predictions: 10406
 % bonnes predictions: 0.8979979288919572


In [ ]:
df_groupe_2['categorie_reelle'].value_counts()

Tomato          4585
Apple           1943
Corn_(maize)    1829
Grape           1805
Potato          1426
Name: categorie_reelle, dtype: int64

In [ ]:
df_groupe_2.groupby('categorie_reelle')['erreur_pred_ModeleC'].sum()

categorie_reelle
Apple           202
Corn_(maize)    313
Grape            46
Potato           41
Tomato          580
Name: erreur_pred_ModeleC, dtype: int64

In [ ]:
print(" % bonnes predictions:", (df_actual_pred['erreur_pred_tot'].count()-df_actual_pred['erreur_pred_tot'].sum())/df_actual_pred['erreur_pred_tot'].count())

 % bonnes predictions: 0.8923856134759846
